In [2]:
import pandas as pd
import xarray as xr
import numpy as np
from glob import glob
from yllib.dataset import myera5

In [3]:
year1=1981
year2=2020

In [4]:
def prepare(vname, year1, year2, ano_flag=True):
    if vname == "sst":
        data = myera5.read_sst(year1, year2).sel(longitude=slice(120,  -70+360), latitude=slice(50,-40))
    if vname == "psfc":
        data = myera5.read_psfc(year1, year2, res="1.0d").sel(longitude=slice(150,-20+360), latitude=slice(70,10))
    if vname in ["t2m"]:
        data = myera5.read_t2m(year1, year2).sel(longitude=slice(-120+360,-113+360), latitude=slice(50,41))
    if vname == "hgt":
        # data = myera5.read_hgt(year1, year2, res="1.0d")#.sel(longitude=slice(150,-20+360), latitude=slice(70,10))
        data = myera5.read_hgt(year1, year2, res="1.0d").sel(longitude=slice(-150+360,-40+360), latitude=slice(70,10))
        data.coords["level"] = [f"z{z:.0f}" for z in data.level.data]
        data = data.to_dataset(dim="level")
    if vname == "wind":
        data = myera5.read_wind(year1, year2).sel(longitude=slice(220,-50+360), latitude=slice(55,20))
    print(data)
    if ano_flag: data = data.groupby(data["time.month"])-data.groupby(data["time.month"]).mean("time")
    return data

def calc_anomaly(data):
    print("calc_anomaly")
    clm = data.groupby(data["time.dayofyear"]).mean()
    ano = data.groupby(data["time.dayofyear"])-clm
    rano = ano / clm
    return ano, clm, rano#.rolling(dayofyear=10, min_periods=2, center=True) 

In [4]:
hgt = prepare('hgt', year1, year2, ano_flag=False)
hgt_ano = calc_anomaly(hgt)

<xarray.Dataset>
Dimensions:    (time: 14610, latitude: 61, longitude: 111)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2020-12-31
  * longitude  (longitude) float64 210.0 211.0 212.0 213.0 ... 318.0 319.0 320.0
  * latitude   (latitude) float64 70.0 69.0 68.0 67.0 ... 13.0 12.0 11.0 10.0
Data variables:
    z200       (time, latitude, longitude) float32 dask.array<chunksize=(31, 61, 111), meta=np.ndarray>
    z500       (time, latitude, longitude) float32 dask.array<chunksize=(31, 61, 111), meta=np.ndarray>
    z850       (time, latitude, longitude) float32 dask.array<chunksize=(31, 61, 111), meta=np.ndarray>
calc_anomaly


In [5]:
hgt = hgt.compute()
hgt_ano = hgt_ano.compute()

In [6]:
# hgt_ano.to_netcdf('data/hgt_daily_anomaliy.nc')
# hgt.to_netcdf('data/hgt_daily.nc')

In [6]:
hgt_ano.to_netcdf('data/hgt_daily_anomaliy_globe.nc')
hgt.to_netcdf('data/hgt_daily_globe.nc')

In [7]:
hgt_ano.to_netcdf('data/hgt_daily_anomaliy_small.nc')
hgt.to_netcdf('data/hgt_daily_small.nc')

## wind speed

In [5]:
wind = prepare('wind', year1, year2, ano_flag=False)
wind_ano, wind_clm, wind_rano = calc_anomaly(wind)

<xarray.Dataset>
Dimensions:    (time: 14610, latitude: 141, longitude: 361)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2020-12-31
  * latitude   (latitude) float64 55.0 54.75 54.5 54.25 ... 20.5 20.25 20.0
  * longitude  (longitude) float64 220.0 220.2 220.5 220.8 ... 309.5 309.8 310.0
Data variables:
    VAR_100UV  (time, latitude, longitude) float32 dask.array<chunksize=(31, 141, 361), meta=np.ndarray>
    power1     (time, latitude, longitude) float64 dask.array<chunksize=(31, 141, 361), meta=np.ndarray>
    power2     (time, latitude, longitude) float64 dask.array<chunksize=(31, 141, 361), meta=np.ndarray>
    power3     (time, latitude, longitude) float64 dask.array<chunksize=(31, 141, 361), meta=np.ndarray>
calc_anomaly


/people/liuy351/software/anaconda3/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 366
  result = blockwise(
/people/liuy351/software/anaconda3/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 366
  result = blockwise(
/people/liuy351/software/anaconda3/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 366
  result = blockwise(
/people/liuy351/software/anaconda3/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 366
  result = blockwise(


In [ ]:
# wind = wind.compute()
# wind_ano = wind_ano.compute()
# wind_clm = wind_clm.compute()
wind_rano = wind_rano.compute()

In [ ]:
# wind_ano.to_netcdf('data/wind_daily_anomaly_us.nc')
# wind_clm.to_netcdf('data/wind_daily_clim_us.nc')
wind_rano.to_netcdf('data/wind_daily_relative_anomaly_us.nc')
# wind.to_netcdf('data/wind_daily_us.nc')